In [1]:
import numpy as np

def calculate_ahp_weights(ahp_matrix):
    # Výpočet vlastních hodnot a vektorů
    eigenvalues, eigenvectors = np.linalg.eig(ahp_matrix)
    # Hledání maximální vlastní hodnoty a odpovídajícího vlastního vektoru
    max_eigenvalue = np.max(eigenvalues)
    max_eigenvector = eigenvectors[:, eigenvalues.argmax()].real
    # Normalizace vlastního vektoru pro získání vah
    weights = max_eigenvector / np.sum(max_eigenvector)
    return weights

def consistency_ratio(ahp_matrix, weights):
    n = ahp_matrix.shape[0]
    # Výpočet matice váženého součtu
    weighted_sum_matrix = np.dot(ahp_matrix, weights)
    # Výpočet lambda_max
    lambda_max = np.mean(weighted_sum_matrix / weights)
    # Výpočet indexu konzistence (CI)
    ci = (lambda_max - n) / (n - 1)
    # Random index (RI) pro náhodnou matici (hodnoty závisí na velikosti matice, příklad pro n=3)
    ri = 0.58  # Tato hodnota se liší podle velikosti matice, pro n=3 je 0.58, pro jiné velikosti viz tabulky RI
    # Výpočet poměru konzistence (CR)
    cr = ci / ri
    return cr


In [2]:
def topsis(decision_matrix, weights):
    # Normalizace matice rozhodování
    norm_matrix = decision_matrix / np.sqrt((decision_matrix**2).sum(axis=0))
    # Vážení normalizované matice
    weighted_matrix = norm_matrix * weights
    # Určení ideálních a neideálních řešení
    ideal_solution = np.max(weighted_matrix, axis=0)
    negative_ideal_solution = np.min(weighted_matrix, axis=0)
    # Výpočet vzdáleností
    distance_to_ideal = np.sqrt(((weighted_matrix - ideal_solution)**2).sum(axis=1))
    distance_to_negative_ideal = np.sqrt(((weighted_matrix - negative_ideal_solution)**2).sum(axis=1))
    # Výpočet relativní blízkosti k ideálnímu řešení
    similarity_to_ideal_solution = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)
    return similarity_to_ideal_solution


In [5]:
# Příklad Saatyho matice (zde použijeme náhodnou matici pro ilustraci)
ahp_matrix = np.array([
    [1,    1/2,  3,    1/4],
    [2,    1,    4,    1/3],
    [1/3,  1/4,  1,    1/5],
    [4,    3,    5,    1]
])

# Příklad matice rozhodování
decision_matrix = np.array([
    [1, 2, 3, 4],  # Varianta 1
    [4, 3, 2, 1],  # Varianta 2
    [3, 3, 3, 3]   # Varianta 3
])

# Výpočet vah kritérií pomocí AHP
weights = calculate_ahp_weights(ahp_matrix)
cr = consistency_ratio(ahp_matrix, weights)

# Kontrola konzistence
if cr < 0.1:
    print("Matice je konzistentní, CR =", cr)
    # Implementace TOPSIS
    similarity_to_ideal_solution = topsis(decision_matrix, weights)
    print("Relativní blízkost k ideálnímu řešení:", similarity_to_ideal_solution)
    # Nejlepší varianta
    best_option_index = np.argmax(similarity_to_ideal_solution)
    print("Nejlepší varianta je:", best_option_index + 1)
else:
    print("Matice není dostatečně konzistentní, CR =", cr)


Matice je konzistentní, CR = 0.06777393837435418
Relativní blízkost k ideálnímu řešení: [0.75632894 0.24367106 0.67318041]
Nejlepší varianta je: 1
